<a href="https://colab.research.google.com/github/lx-jdar/progra-concurrente/blob/development/tp1/go/tp1_goland.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo práctico 1: GOLANG

Instalacion de golang en Colab

In [ ]:
!add-apt-repository ppa:longsleep/golang-backports -y
!apt update
!apt install golang-go
%env GOPATH=/root/go
!go version

Creación del Archivo GO

In [9]:
%%writefile prueba.go
package main

import (
	"fmt"
	"os"
	"syscall"
	"time"
)

var nodes map[string]string = map[string]string{
	"A": "CB", "B": "ED", "C": "F",
	"D": "", "E": "HG", "F": ""}

func fork() (uintptr, error) {
	// fork off the parent process
	ret, ret2, errno := syscall.RawSyscall(syscall.SYS_FORK, 0, 0, 0)
	// failure while forking
  if errno != 0 {
		return uintptr(errno), fmt.Errorf("fork failed with error %s", errno.Error())
	}

	// failure
	if ret2 < 0 {
		return ret2, fmt.Errorf("fork failed with pid %d", ret2)
	}

	return ret, nil
}

func createProcess(letter string) uintptr {
	//fmt.Println("This is the process " + letter)

	for i := 0; i < len(nodes[letter]); i++ {

		id, _ := fork()
		if id == 0 {
			node := string(nodes[letter][i])
			fmt.Printf("PID_Child: %d Padre: %d Node: %s\n", os.Getpid(), os.Getppid(), node)
			if len(nodes[letter]) != 0 {
				createProcess(node)
			}
			time.Sleep(10 * time.Second)
			return id
		}
		time.Sleep(1 * time.Second)
	}

	return 1
}

func main() {
	fmt.Printf("PPID_INIT: %d Padre: %d Node: A\n", os.Getpid(), os.Getppid())
	if createProcess("A") == 1 {
		time.Sleep(10 * time.Second)
	}
}



Overwriting prueba.go


Compilacion del Archivo GO

In [10]:
!go build prueba.go

Ejecución del Programa

In [11]:
!./prueba

PPID_INIT: 4299 Padre: 398 Node: A
PID_Child: 4303 Padre: 4299 Node: C
PID_Child: 4304 Padre: 4303 Node: F
PID_Child: 4309 Padre: 4299 Node: B
PID_Child: 4310 Padre: 4309 Node: E
PID_Child: 4311 Padre: 4310 Node: H
PID_Child: 4316 Padre: 4309 Node: D
PID_Child: 4317 Padre: 4310 Node: G


Ejecucion del Programa Background logueando a Archivo

In [17]:
!nohup ./prueba 1>salidaGO 2>/dev/null &

Visualizacion de los Procesos

In [16]:
!ps -axf | grep prueba

   5596 ?        S      0:00  |       \_ /bin/bash -c ps -axf | grep prueba
   5598 ?        S      0:00  |           \_ grep prueba
   5468 ?        S      0:00 ./prueba
   5474 ?        S      0:00 ./prueba
   5573 ?        Sl     0:00 ./prueba
   5577 ?        S      0:00  \_ ./prueba
   5578 ?        S      0:00  |   \_ ./prueba
   5583 ?        S      0:00  \_ ./prueba
   5584 ?        S      0:00      \_ ./prueba
   5585 ?        S      0:00      |   \_ ./prueba
   5591 ?        S      0:00      |   \_ ./prueba
   5590 ?        S      0:00      \_ ./prueba


Visualizacion de log del Programa GO a un Archivo

In [18]:
!cat salidaGO && echo

PPID_INIT: 5876 Padre: 1 Node: A
PID_Child: 5880 Padre: 5876 Node: C
PID_Child: 5881 Padre: 5880 Node: F
PID_Child: 5886 Padre: 5876 Node: B
PID_Child: 5887 Padre: 5886 Node: E
PID_Child: 5888 Padre: 5887 Node: H
PID_Child: 5893 Padre: 5886 Node: D
PID_Child: 5894 Padre: 5887 Node: G

